In [2]:
#下載csv檔
import requests
csv_url='https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=CSV'
res=requests.get(url=csv_url)
if res.ok:
    print('下載成功')
else:
    print('下載失敗')

下載成功


In [ ]:
import io
import csv
csv_str:str=res.text #csv_str是包含csv資料的字串
csv_file=io.StringIO(csv_str) #將csv_str轉為類文件對象
dict_reader=csv.DictReader(csv_file) #將每行的資料轉為dict
#print(type(dict_reader))
csv_data=list(dict_reader) #將資料轉為list
for row in csv_data: #印出資料
    print(row)
#檢查資料類型
#保證一定有資料
#過濾dict的key

In [4]:
from datetime import datetime
from pydantic import BaseModel,Field,RootModel
class Site(BaseModel):
    站點名稱:str=Field(alias='sitename')
    城鎮:str=Field(alias='county')
    aqi:int=Field
    狀態:str=Field(alias='status')
    日期:datetime=Field(alias='datacreationdate')
#適用RootModel的是list
#適用BaseModel的是dict
class Csvroot(RootModel):
    root:list[Site]
    def __iter__(self):
        return iter(self.root)
    
    def __getitem__(self, item):
        return self.root[item]

In [5]:
csv_root=Csvroot.model_validate(csv_data)
type(csv_root)

__main__.Csvroot

In [ ]:
for site in csv_root:
    print(type(site))

In [ ]:
for site in csv_root:
    print(site)

In [ ]:
for site in csv_root:
    print(site.model_dump())

In [9]:
with open('new_aqi.csv', mode='w', encoding='utf-8', newline='') as file:
    #dict_writer=csv.DictWriter(file, fieldnames=['站點名稱', '城鎮', 'aqi', '狀態', '日期'])
    dict_writer=csv.DictWriter(file, fieldnames=list(Site.model_fields.keys()))
    dict_writer.writeheader()
    for site in csv_root:
        dict_writer.writerow(site.model_dump())
print('存檔成功')


存檔成功


In [10]:
#type(Site.model_fields)
#快速寫入fieldnames
list(Site.model_fields.keys())

['站點名稱', '城鎮', 'aqi', '狀態', '日期']

In [11]:
#寫json檔
json_str=csv_root.model_dump_json()
with open('new_aqi.json', mode='w', encoding='utf-8') as jsonFile:
    jsonFile.write(json_str)

In [13]:
import pandas
from pandas import DataFrame
aqi:DataFrame=DataFrame(data=csv_root.model_dump())
aqi

,站點名稱,城鎮,aqi,狀態,日期
0,屏東(枋山),屏東縣,21,良好,2024-07-03 21:00:00
1,大甲（日南國小）,臺中市,27,良好,2024-07-03 21:00:00
2,新北(樹林),新北市,31,良好,2024-07-03 21:00:00
3,屏東（琉球）,屏東縣,22,良好,2024-07-03 21:00:00
4,臺南（麻豆）,臺南市,23,良好,2024-07-03 21:00:00
...,...,...,...,...,...
995,頭份,苗栗縣,24,良好,2024-07-03 10:00:00
996,新竹,新竹市,14,良好,2024-07-03 10:00:00
997,竹東,新竹縣,16,良好,2024-07-03 10:00:00
998,湖口,新竹縣,29,良好,2024-07-03 10:00:00
